<a href="https://colab.research.google.com/github/balnarendrasapa/cybersecurity-ner/blob/master/Fine_tune_for_Cybersecurity_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!wget https://github.com/balnarendrasapa/cybersecurity-ner/raw/master/MITRE.zip

--2023-11-20 22:26:59--  https://github.com/balnarendrasapa/cybersecurity-ner/raw/master/MITRE.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/balnarendrasapa/cybersecurity-ner/master/MITRE.zip [following]
--2023-11-20 22:26:59--  https://raw.githubusercontent.com/balnarendrasapa/cybersecurity-ner/master/MITRE.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 238392 (233K) [application/zip]
Saving to: ‘MITRE.zip’

MITRE.zip           100%[===================>] 232.80K  --.-KB/s    in 0.03s   

2023-11-20 22:27:00 (8.78 MB/s) - ‘MITRE.zip’ saved [238392/238392]



In [4]:
!unzip MITRE.zip

Archive:  MITRE.zip
   creating: MITRE/
  inflating: MITRE/test.txt          
  inflating: MITRE/train.txt         
  inflating: MITRE/valid.txt         


In [5]:
!pip -q install accelerate transformers datasets seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.3 MB/s eta 0:00:00


# Login to the Huggingface

- get huggingface write access token from huggingface account

In [63]:
from huggingface_hub import notebook_login

notebook_login()

# Select the base model and task and batch size

In [64]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

# Load the train.txt, test.txt and valid.txt using datasets library.

In [65]:
from datasets import load_dataset
raw_dataset = load_dataset("text", data_files={"train": "MITRE/train.txt", "test": "MITRE/test.txt", "valid": "MITRE/valid.txt"})

In [66]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 71001
    })
    test: Dataset({
        features: ['text'],
        num_rows: 20017
    })
    valid: Dataset({
        features: ['text'],
        num_rows: 20342
    })
})

# Dataset Preprocessing

- We need to convert the raw dataset to huggingface compatible dataset.
- This is general preprocessing. In order to use it for fine-tuning, you need to again preprocess it according to the model's requirements

### The function below seperates sentences and tags from the raw dataset

In [67]:
def seperate_sentence_tags(dataset):
    sentences = []
    tags = []
    sentence = []
    tag = []
    for i in dataset:
        if i["text"] != "":
            word, word_tag = i["text"].split("\t")
            sentence += [word]
            tag += [word_tag]
            if word == ".":
                sentences += [sentence]
                tags += [tag]
                sentence = []
                tag = []

    return sentences, tags

### The function below finds unique tags and converts them into a python dictionary

In [68]:
def unique_tags_dict(tags):
    unique_tags = set()
    for i in tags:
        for j in i:
            unique_tags.add(j)

    tags_dict = dict()
    j = 0
    unique_tags = sorted(list(unique_tags))
    for i in unique_tags:
        tags_dict[i] = j
        j += 1

    return unique_tags, tags_dict

### The function below converts tags to its assigned numbers in each sentence

In [69]:
def tag_to_num(tags_dict, tags):
    tags_nums = []
    for i in tags:
        temp = [tags_dict[j] for j in i]
        tags_nums += [temp]
        temp = []
    return tags_nums

### The function below creates a huggingface compatible dataset.

- It creates a dataset with the help of the functions above. it takes in sentences, tags_num (which is nothing but tags converted to numbers), unique_tags(list of unique tags in the dataset).

In [70]:
from datasets import Dataset, Features, ClassLabel, Value, Sequence

def dataset_hf(sentences, tags_nums, unique_tags):
    dataset = Dataset.from_dict({
        "id": list(range(len(sentences))),
        "tokens": sentences,
        "ner_tags": tags_nums
        },
        features = Features({
            "id": Value(dtype='string'),
            "ner_tags" : Sequence(ClassLabel(names = list(unique_tags))),
            "tokens": Sequence(Value(dtype = "string"))
        }
    ))
    return dataset

### Combine the functions above into a single function

- The function below returns a huggingface compatible dataset by using all the above functions defined to pre-process the dataset

In [71]:
def dataset_preprocess(dataset):
    sentences, tags = seperate_sentence_tags(dataset)
    unique_tags, tags_dict = unique_tags_dict(tags)
    tags_nums = tag_to_num(tags_dict, tags)
    return dataset_hf(sentences, tags_nums, unique_tags)

In [72]:
raw_dataset["test"][0]

{'text': 'Why\tO'}

### Preprocess the train, test, validation dataset

In [73]:
train_dataset = dataset_preprocess(raw_dataset["train"])
test_dataset = dataset_preprocess(raw_dataset["test"])
validation_dataset = dataset_preprocess(raw_dataset["valid"])

### Convert them into a single whole dataset dictionary

In [74]:
from datasets import DatasetDict


datasets = DatasetDict({
    'train':train_dataset,
    'test': test_dataset,
    'validation':validation_dataset
})

datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2664
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 717
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 785
    })
})

### Push it to the huggingface

- here `bnsapa/cybersecurity-ner` is repository from my huggingface account.

In [ ]:
datasets.push_to_hub("bnsapa/cybersecurity-ner")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/880 [00:00<?, ?B/s]

### You can import the preprocessed dataset that is pushed to huggingface by below code

In [75]:
from datasets import load_dataset

datasets = load_dataset("bnsapa/cybersecurity-ner")

# See Random datapoints in the dataset

- the function below prints the random elements from the dataset

In [76]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [77]:
show_random_elements(datasets["train"], num_examples = 5)

,id,tokens,ner_tags
0,2619,"[The, malware, was, able, to, perform, overlay, attacks, and, become, the, default, SMS, app, through, the, abuse, of, the, Accessibility, Service, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
1,456,"[We, expect, it, to, churn, out, new, variants, with, even, more, sophisticated, techniques, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
2,102,"[], com, hxxp, :, //mailsa-wqq, [, .]","[I-Indicator, I-Indicator, B-Indicator, I-Indicator, I-Indicator, I-Indicator, I-Indicator]"
3,1126,"[According, to, Google, ,, whom, we, have, contacted, to, alert, about, our, discoveries, ,, nearly, 25, variants, of, this, spyware, were, uploaded, on, Google, Play, Store, .]","[O, O, B-Organization, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-System, I-System, I-System, O]"
4,1003,"[Within, this, time, period, ,, we, identified, close, to, 300, samples, belonging, to, this, family, (, all, sample, hashes, are, listed, in, the, Appendix, ), .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"


# See an example and dataset's features

In [78]:
print(datasets["train"][1082])

datasets["train"].features

{'id': '1082', 'tokens': ['RuMMS', 'samples', 'and', 'C2', 'servers', 'Figure', '8', 'shows', 'how', 'these', 'samples', ',', 'C2', 'servers', 'and', 'hosting', 'websites', 'are', 'related', 'to', 'each', 'other', ',', 'including', 'when', 'they', 'were', 'compiled', 'or', 'observed', '.'], 'ner_tags': [1, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]}


{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['B-Indicator', 'B-Malware', 'B-Organization', 'B-System', 'B-Vulnerability', 'I-Indicator', 'I-Malware', 'I-Organization', 'I-System', 'I-Vulnerability', 'O'], id=None), length=-1, id=None)}

# Load the pretrained model

In [79]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Preprocessing the dataset for the pretrained model

- We need to preprocess the model according the pretrained model's requirements.
- Get the tokenizer for the model

In [80]:
import transformers

### Tokenizer converts the words into numbers that are understandable by the model

In [81]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [101, 7592, 1010, 2023, 2003, 2028, 6251, 999, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [82]:
tokenizer(["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."], is_split_into_words=True)

{'input_ids': [101, 7592, 1010, 2023, 2003, 2028, 6251, 3975, 2046, 2616, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [83]:
example = datasets["train"][4]
print(example["tokens"])

['The', 'authors', 'are', 'trying', 'to', 'latch', 'onto', 'the', 'popularity', 'of', 'the', 'Super', 'Mario', 'Run', 'game', 'to', 'target', 'eagerly', 'waiting', 'Android', 'users', '.']


# Accounting for splitting of words by tokenizer

- Sometimes tokenizer splits the words like `sheepmeat` is split into three words `sheap`, `##me`, `##at`.
- we need to add ner tags for these words too.
- This is done by the code in this section.

In [84]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'the', 'authors', 'are', 'trying', 'to', 'latch', 'onto', 'the', 'popularity', 'of', 'the', 'super', 'mario', 'run', 'game', 'to', 'target', 'eagerly', 'waiting', 'android', 'users', '.', '[SEP]']


In [85]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

(22, 24)

In [86]:
print(tokenized_input.word_ids())

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, None]


### Set the labels of all special tokens to -100 (the index that is ignored by PyTorch)

In [87]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

24 24


In [88]:
label_all_tokens = True

# Write custom function to tokenize and align labels

In [89]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [90]:
tokenize_and_align_labels(datasets['train'][:5])

{'input_ids': [[101, 3565, 7986, 2448, 15451, 8059, 1001, 1016, 1516, 2852, 9314, 17364, 9350, 27911, 2015, 2293, 7986, 1998, 20421, 1010, 2021, 2061, 2079, 15451, 8059, 6048, 1012, 102], [101, 1037, 2261, 2420, 2067, 1010, 2057, 2626, 2055, 2019, 11924, 28791, 2099, 23445, 8349, 20540, 2004, 1996, 3565, 7986, 2448, 2208, 2005, 11924, 1012, 102], [101, 2057, 2031, 2179, 2178, 6013, 1997, 15451, 8059, 20540, 2004, 1996, 3565, 7986, 2448, 11924, 10439, 1010, 1998, 2023, 2051, 2009, 2038, 2579, 1996, 2433, 1997, 2852, 9314, 17364, 9350, 1006, 6556, 3229, 23445, 1007, 1012, 102], [101, 6947, 8400, 2626, 2055, 1996, 2852, 9314, 17364, 9350, 2217, 1011, 8209, 2007, 1996, 20421, 2175, 10439, 2067, 1999, 2251, 2355, 1025, 1996, 4489, 2182, 2003, 2008, 2045, 2003, 2053, 2208, 2443, 1999, 1996, 24391, 7427, 1012, 102], [101, 1996, 6048, 2024, 2667, 2000, 25635, 3031, 1996, 6217, 1997, 1996, 3565, 7986, 2448, 2208, 2000, 4539, 17858, 3403, 11924, 5198, 1012, 102]], 'attention_mask': [[1, 1, 1, 1,

In [91]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

### See the ner labels

In [92]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

['B-Indicator',
 'B-Malware',
 'B-Organization',
 'B-System',
 'B-Vulnerability',
 'I-Indicator',
 'I-Malware',
 'I-Organization',
 'I-System',
 'I-Vulnerability',
 'O']

# Fine-tune the model

In [93]:
from transformers import AutoConfig

In [94]:
_, tags = seperate_sentence_tags(raw_dataset["train"])
_, tags_dict = unique_tags_dict(tags)

In [95]:
tags_dict

{'B-Indicator': 0,
 'B-Malware': 1,
 'B-Organization': 2,
 'B-System': 3,
 'B-Vulnerability': 4,
 'I-Indicator': 5,
 'I-Malware': 6,
 'I-Organization': 7,
 'I-System': 8,
 'I-Vulnerability': 9,
 'O': 10}

In [96]:
ids_dict = {value:key for key, value in tags_dict.items()}

In [97]:
ids_dict

{0: 'B-Indicator',
 1: 'B-Malware',
 2: 'B-Organization',
 3: 'B-System',
 4: 'B-Vulnerability',
 5: 'I-Indicator',
 6: 'I-Malware',
 7: 'I-Organization',
 8: 'I-System',
 9: 'I-Vulnerability',
 10: 'O'}

In [98]:
config = AutoConfig.from_pretrained(model_checkpoint, num_labels=len(label_list), label2id=tags_dict, id2label=ids_dict)

In [99]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, config = config)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Instantiate TrainingArguments object

- This objects holds all the training hyperparameters

In [100]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"cybersecurity-ner",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=8,
    weight_decay=0.01,
    push_to_hub=True,
)

In [101]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [102]:
from datasets import load_metric

metric = load_metric("seqeval")

In [103]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'System': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [104]:
print(labels)

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-System', 'I-System', 'I-System', 'O', 'O', 'O', 'O', 'O', 'B-System', 'O', 'O']


### Write function to calculate metrics

In [105]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

### Create a Trainer object to train the model

In [106]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

### Train the model

In [107]:
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.235000,0.681042,0.764780,0.720486,0.933544
2,No log,0.195180,0.783037,0.749057,0.765670,0.945671
3,0.211700,0.191017,0.782424,0.753145,0.767505,0.947597
4,0.211700,0.189538,0.776535,0.807547,0.791737,0.951899
5,0.211700,0.208149,0.784735,0.792138,0.788419,0.951203
6,0.050500,0.213356,0.771611,0.800000,0.785549,0.951080
7,0.050500,0.227449,0.791411,0.770755,0.780946,0.949891
8,0.050500,0.223270,0.785625,0.790566,0.788088,0.951366


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=1336, training_loss=0.10545385240794655, metrics={'train_runtime': 303.8856, 'train_samples_per_second': 70.132, 'train_steps_per_second': 4.396, 'total_flos': 709028875718064.0, 'train_loss': 0.10545385240794655, 'epoch': 8.0})

In [108]:
trainer.evaluate()

{'eval_loss': 0.22326971590518951,
 'eval_precision': 0.785625,
 'eval_recall': 0.7905660377358491,
 'eval_f1': 0.7880877742946708,
 'eval_accuracy': 0.9513664112754537,
 'eval_runtime': 3.0606,
 'eval_samples_per_second': 256.489,
 'eval_steps_per_second': 16.337,
 'epoch': 8.0}

# See how it is working on Test dataset

In [109]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'Indicator': {'precision': 0.7929039722329349,
  'recall': 0.888504753673293,
  'f1': 0.8379865498267781,
  'number': 2314},
 'Malware': {'precision': 0.7255639097744361,
  'recall': 0.6380165289256199,
  'f1': 0.6789797713280562,
  'number': 605},
 'Organization': {'precision': 0.65,
  'recall': 0.4315352697095436,
  'f1': 0.5187032418952617,
  'number': 241},
 'System': {'precision': 0.39635535307517084,
  'recall': 0.4371859296482412,
  'f1': 0.4157706093189964,
  'number': 398},
 'Vulnerability': {'precision': 0.7090909090909091,
  'recall': 0.8297872340425532,
  'f1': 0.764705882352941,
  'number': 47},
 'overall_precision': 0.7300873246890712,
 'overall_recall': 0.7653259361997226,
 'overall_f1': 0.7472914409534128,
 'overall_accuracy': 0.9340902685509522}

# Metrics

- Accuracy on the test dataset is nearly 93 %.
- The F1-Score is nearly 74 %.

# Push the model to Huggingface

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1700352893.f88aac7ea754.772.0:   0%|          | 0.00/9.07k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1700353196.f88aac7ea754.772.1:   0%|          | 0.00/560 [00:00<?, ?B/s]

'https://huggingface.co/bnsapa/cybersecurity-ner/tree/main/'